In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
from pprint import pprint
import zipcodes

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2017)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_housing = c.acs5.get(("NAME", "B19013_001E", "B25058_001E", "B25064_001E", "B25077_001E", "B25088_002E"), {'for': 'state:*'}) #'for': 'zip code tabulation area:*' ) 

# Convert to DataFrame
census_housing = pd.DataFrame(census_housing)

# Column Reordering
census_housing = census_housing.rename(columns={
                                      "B19013_001E": "Household Income",
                                      "B25058_001E": "Median Contract Rent",
                                      "B25064_001E": "Median Gross Rent",
                                      "B25077_001E": "Median Home Value",
                                      "B25088_002E": "Median Monthly Owner Costs",
                                      "NAME": "Name","state": "State"})  #"zip code tabulation area": "Zipcode"


# Final DataFrame
census_housing = census_housing[["State", "Name", "Household Income", "Median Contract Rent", "Median Gross Rent", "Median Home Value",
                       "Median Monthly Owner Costs"]]

census_housing.head()

NameError: name 'census_data' is not defined

In [ ]:
# Read in the csv containing state happiness rates
happiness = pd.read_csv("./happiness.csv")
happiness.head()